# Análise Estatística dos Acidentes de Trânsito

## Projeto da disciplina **SSC0957 - Práticas em Ciência de Dados II**

- Alexandre Eduardo de Souza Jesus - alexandre_souza@usp.br - 12559506

- Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472

- Gustavo Silva de Oliveira - guspfc03@usp.br - 12567231

<img src="pictures.gif" style="width: 200px;"/>

---


## 6. Análise Estatística dos Dados

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

In [3]:
prf_rain_data = pd.read_csv("processed_data/prf_rain_data.csv", index_col=0)
prf_rain_data.columns
prf_rain_data = prf_rain_data[['rain','temperature_2m','apparent_temperature','relativehumidity_2m','cloudcover','windspeed_10m']]

In [4]:
prf_not_rain_data = pd.read_csv("processed_data/prf_not_rain_data.csv", index_col=0)
prf_not_rain_data = prf_not_rain_data[['rain','temperature_2m','apparent_temperature','relativehumidity_2m','cloudcover','windspeed_10m']]

In [5]:
# testando normalidade das variáveis com o teste shapiro-wilk
# H0 (hipotese nula) - os dados seguem uma distribuição normal

alpha = 0.05

for column in prf_rain_data.columns:
    statistic, p_value = stats.shapiro(prf_rain_data[column])
    if p_value > alpha:
        print(f"{column}: os dados são normais")
    else:
        print(f"{column}: os dados não são normais")

rain: os dados não são normais
temperature_2m: os dados são normais
apparent_temperature: os dados não são normais
relativehumidity_2m: os dados não são normais
cloudcover: os dados não são normais
windspeed_10m: os dados não são normais


In [6]:
alpha = 0.05

for column in prf_not_rain_data.columns:
    statistic, p_value = stats.shapiro(prf_not_rain_data[column])
    if p_value > alpha:
        print(f"{column}: os dados são normais")
    else:
        print(f"{column}: os dados não são normais")

rain: os dados não são normais
temperature_2m: os dados não são normais
apparent_temperature: os dados não são normais
relativehumidity_2m: os dados não são normais
cloudcover: os dados não são normais
windspeed_10m: os dados não são normais


In [7]:
# como a normalidade foi rejeitada, utiliza-se o teste não-paramétrico de Mann-Whitney para verificar 
# a diferença entre os grupos

alpha = 0.05

statistic, p_value = stats.mannwhitneyu(prf_rain_data['rain'], prf_not_rain_data['rain'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

há diferença significativa entre as variáveis (1.2329597100927212e-132)


In [8]:
statistic, p_value = stats.mannwhitneyu(prf_rain_data['temperature_2m'], prf_not_rain_data['temperature_2m'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

há diferença significativa entre as variáveis (7.172223211328207e-07)


In [9]:
statistic, p_value = stats.mannwhitneyu(prf_rain_data['apparent_temperature'], prf_not_rain_data['apparent_temperature'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

não há diferença significativa entre as variáveis (0.14641530295557445)


In [10]:
statistic, p_value = stats.mannwhitneyu(prf_rain_data['relativehumidity_2m'], prf_not_rain_data['relativehumidity_2m'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

há diferença significativa entre as variáveis (1.2481311341241215e-56)


In [11]:
statistic, p_value = stats.mannwhitneyu(prf_rain_data['cloudcover'], prf_not_rain_data['cloudcover'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

há diferença significativa entre as variáveis (3.963067035891679e-129)


In [12]:
statistic, p_value = stats.mannwhitneyu(prf_rain_data['windspeed_10m'], prf_not_rain_data['windspeed_10m'])

if p_value > alpha:
    print(f"não há diferença significativa entre as variáveis ({p_value})")
else:
    print(f"há diferença significativa entre as variáveis ({p_value})")

há diferença significativa entre as variáveis (0.000606321895064651)


In [13]:
prf_rain_data.describe()

,rain,temperature_2m,apparent_temperature,relativehumidity_2m,cloudcover,windspeed_10m
count,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000
mean,0.904934,21.723612,23.884846,84.630837,81.125110,8.910573
std,1.523465,4.237671,5.584589,12.947197,24.388504,5.102003
min,0.000000,9.100000,5.800000,35.000000,2.000000,0.000000
25%,0.100000,18.950000,20.400000,78.000000,66.000000,5.200000
50%,0.300000,21.900000,24.400000,88.000000,95.000000,7.900000
75%,1.100000,24.500000,27.700000,94.000000,100.000000,11.550000
max,18.300000,34.800000,39.000000,100.000000,100.000000,45.500000


In [14]:
prf_not_rain_data.describe()

,rain,temperature_2m,apparent_temperature,relativehumidity_2m,cloudcover,windspeed_10m
count,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000
mean,0.192952,22.547137,24.015242,73.256388,45.554185,9.527489
std,0.687469,5.120949,6.489959,18.243512,34.270380,4.987270
min,0.000000,2.400000,-1.300000,17.000000,0.000000,0.400000
25%,0.000000,19.300000,19.800000,61.000000,15.000000,5.800000
50%,0.000000,22.900000,24.700000,76.000000,40.000000,8.600000
75%,0.100000,26.000000,28.800000,88.000000,75.000000,12.600000
max,9.000000,35.600000,39.900000,100.000000,100.000000,35.600000
